In [20]:
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

ip_D = "192.168.178.27"
col_names = ['website-index', 'time', 'src', 'dst','packet size', 'protocol',  'info']

In [61]:
# TODO: Modify this every time 
out_prefix = "collection/" + "df_http" + "/"
if not os.path.exists(out_prefix):
    os.mkdir(out_prefix)    

def clean(df):
    # HTTP, TLSv1.3, UDP, TLSv1.3, TLSv1, TLSv1.2, DNS, ICMP
    # the only good ones are probably HTTP, TLS, UDP, TCP
    df_all = df
    df_http = df[df.protocol == "HTTP"]
    df_tls = df.loc[df.protocol.isin(["TLSv1.3", "TLSv1", "TLSv1.2"])]
    df_tcp = df[df.protocol == "TCP"]
    df_udp = df[df.protocol == "UDP"]
    
    # Subsequence generated through javscript ay
    df_http_tls = pd.concat([df_http, df_tls])
    df_http_tls_tcp = pd.concat([df_http, df_tls, df_tcp])
    df_http_tls_tcp_udp = pd.concat([df_http, df_tls, df_tcp, df_udp])
    df_http_tls_udp = pd.concat([df_http, df_tls, df_udp])
    df_http_tcp = pd.concat([df_http, df_tcp])
    df_http_tcp_udp = pd.concat([df_http, df_tcp, df_udp])
    df_http_udp = pd.concat([df_http, df_udp])
    df_tls_tcp = pd.concat([df_tls, df_tcp])
    df_tls_tcp_udp = pd.concat([df_tls, df_tcp, df_udp])
    df_tls_udp = pd.concat([df_tls, df_udp])
    df_tcp_udp = pd.concat([df_tcp, df_udp])
  
    
    # TODO: Modify this every time
    df_selection = df_http
    
    
    
    print(df_selection)
    
    # If a data frame is empty, don't do the sorting/time zero-ing stuff below
    if df_selection.shape[0] == 0: 
        return df_selection.drop(['src', 'dst', 'info', 'protocol'], axis=1)        
    
    df_selection_sorted = df_selection.sort_values(by='time')
    
    start_time = df_selection_sorted['time'].iloc[0] # Produces big errors if empty
    df_selection_sorted['time'] = df_selection_sorted['time'].apply(lambda t : t - start_time)
    df_selection_sorted['time'] = df_selection_sorted['time'].round(5)
    
    return df_selection_sorted.drop(['src', 'dst', 'info', 'protocol'], axis=1)

In [62]:
import os

targets = ["youtube.com", "yahoo.com", "facebook.com", "reddit.com", "instructure.com", "stackoverflow.com", "linkedin.com", "irs.gov", "nytimes.com", "cnn.com", "salesforce.com", "okta.com", "wikipedia.org", "imgur.com", "dropbox.com", "etsy.com", "hulu.com", "quizlet.com"]
targets_name = ["youtube", "yahoo", "facebook", "reddit", "instructure", "stackoverflow", "linkedin", "irs", "nytimes", "cnn", "salesforce", "okta", "wikipedia", "imgur", "dropbox", "etsy", "hulu", "quizlet"]
day1ext = ["1589833992.csv", "1589834155.csv", "1589834318.csv", "1589834481.csv", "1589834644.csv", "1589834807.csv", "1589834970.csv", "1589835133.csv", "1589835296.csv", "1589835459.csv"]
day2ext = ["1589836286.csv", "1589836449.csv", "1589836611.csv", "1589836774.csv", "1589836938.csv", "1589837101.csv", "1589837264.csv", "1589837427.csv", "1589837590.csv", "1589837753.csv"]
day3ext = ["1589927378.csv", "1589927541.csv", "1589927704.csv", "1589927867.csv", "1589928030.csv", "1589928193.csv", "1589928356.csv", "1589928519.csv", "1589928682.csv", "1589928845.csv"]
m = {"day_one": day1ext, "day_two": day2ext, "day_three": day3ext}
output = {"day_one": "Day1-parsed-ondevice.csv", "day_two": "Day2-parsed-ondevice.csv", "day_three": "Day3-parsed-ondevice.csv"}
raw_output = {"day_one": "Day1-raw-ondevice.csv", "day_two": "Day2-raw-ondevice.csv", "day_three": "Day3-raw-ondevice.csv"}



for day in m:
    mdf = pd.DataFrame(columns = ['website-index', 'time', 'direction', 'packet size'])
    raw_mdf = pd.DataFrame(columns = col_names)
    
    for ext in m[day]:
        for i in range(len(targets)):
            target_name = targets_name[i]
            file_name = './raw_data/' + day + '/' + targets[i] + "_" + ext
            
            # Before reading the csv, we should fix up the `info` column to escape commas 
            # We do this by inserting a " after the 6th comma, and " at the end 
            with open (file_name, 'r') as file:
                filedata = file.read()
                # MAGIC 
                filedata_regexp1 = re.sub(r"(^(?:[^\,]+\,){4})((?:[^\,]+\,){2})(.*Destination unreachable.*\n)", r"\1\3", filedata, flags=re.M)

                new_filedata = re.sub(r"(^(?:[^\,]+\,){6})(.*)", r"\1^\2^", filedata_regexp1, flags=re.M)
                
    
            fixed_filename = './fixed_data/' + targets[i] + "_" + ext + "_fixed.csv"
            with open(fixed_filename, 'w') as file: 
                file.write(new_filedata)
            
            # NEED quotechar as ^
            df = pd.read_csv(fixed_filename, header=0, names=col_names, error_bad_lines=False, warn_bad_lines=False, quotechar="^")
            print(file_name + "_fixed.csv read" )
            # raw_mdf = pd.concat([raw_mdf, df]) # No need to do this everytime
            
            df['website-index'] = target_name
            df.insert(2, 'direction', (df['src'] == ip_D).astype(int))
            cdf = clean(df)
            mdf = pd.concat([mdf, cdf])
    #raw_mdf.to_csv(out_prefix + raw_output[day], index=False) # No need to do this everytime
    mdf.to_csv(out_prefix + output[day], index=False)




./raw_data/day_one/youtube.com_1589833992.csv_fixed.csv read
    website-index      time  direction              src              dst  \
0         youtube  0.000322          0   216.58.208.110   192.168.178.27   
7         youtube  1.148118          1   192.168.178.27    172.217.17.54   
9         youtube  1.154713          1   192.168.178.27  172.217.168.193   
10        youtube  1.163660          0    172.217.17.54   192.168.178.27   
11        youtube  1.163664          0    172.217.17.54   192.168.178.27   
12        youtube  1.164740          1   192.168.178.27    172.217.17.54   
13        youtube  1.169712          0  172.217.168.193   192.168.178.27   
14        youtube  1.169716          0  172.217.168.193   192.168.178.27   
15        youtube  1.169915          1   192.168.178.27  172.217.168.193   
16        youtube  1.170124          0  172.217.168.193   192.168.178.27   
17        youtube  1.171417          1   192.168.178.27  172.217.168.193   
18        youtube  1.172774

./raw_data/day_one/instructure.com_1589833992.csv_fixed.csv read
     website-index      time  direction             src             dst  \
3      instructure  0.007502          1  192.168.178.27  172.217.19.205   
14     instructure  0.024431          0  172.217.19.205  192.168.178.27   
15     instructure  0.024489          0  192.168.178.67  192.168.178.27   
16     instructure  0.024490          0  192.168.178.67  192.168.178.27   
20     instructure  0.024859          1  192.168.178.27  172.217.19.205   
21     instructure  0.025239          1  192.168.178.27  172.217.19.205   
32     instructure  0.051247          0  172.217.19.205  192.168.178.27   
33     instructure  0.051252          0  172.217.19.205  192.168.178.27   
34     instructure  0.051590          0  172.217.19.205  192.168.178.27   
35     instructure  0.051678          1  192.168.178.27  172.217.19.205   
40     instructure  0.077529          1  192.168.178.27  172.217.19.205   
76     instructure  0.199531       

     website-index      time  direction             src             dst  \
8          nytimes  0.019982          1  192.168.178.27   172.217.17.77   
21         nytimes  0.041362          0   172.217.17.77  192.168.178.27   
22         nytimes  0.041366          0   172.217.17.77  192.168.178.27   
25         nytimes  0.043318          1  192.168.178.27   172.217.17.77   
33         nytimes  0.063228          1  192.168.178.27   172.217.17.77   
39         nytimes  0.072519          0  192.168.178.67  192.168.178.27   
40         nytimes  0.072523          0  192.168.178.67  192.168.178.27   
53         nytimes  0.084707          0   172.217.17.77  192.168.178.27   
54         nytimes  0.085129          1  192.168.178.27   172.217.17.77   
55         nytimes  0.085384          1  192.168.178.27   172.217.17.77   
60         nytimes  0.111336          0   172.217.17.77  192.168.178.27   
61         nytimes  0.111528          0   172.217.17.77  192.168.178.27   
62         nytimes  0.111

    website-index      time  direction             src              dst  \
5       wikipedia  0.007919          0  192.168.178.67   192.168.178.27   
6       wikipedia  0.008366          1  192.168.178.27   216.58.211.109   
7       wikipedia  0.009522          0  192.168.178.67   192.168.178.27   
15      wikipedia  0.023261          0  216.58.211.109   192.168.178.27   
16      wikipedia  0.023681          1  192.168.178.27   216.58.211.109   
17      wikipedia  0.023933          1  192.168.178.27   216.58.211.109   
33      wikipedia  0.047206          0  216.58.211.109   192.168.178.27   
34      wikipedia  0.047485          0  216.58.211.109   192.168.178.27   
35      wikipedia  0.047488          0  216.58.211.109   192.168.178.27   
36      wikipedia  0.047976          1  192.168.178.27   216.58.211.109   
42      wikipedia  0.073067          1  192.168.178.27   216.58.211.109   
81      wikipedia  0.209266          0  192.168.178.67   192.168.178.27   
82      wikipedia  0.2092

    website-index      time  direction             src             dst  \
1         dropbox  0.005385          1  192.168.178.27  216.58.211.109   
5         dropbox  0.019421          0  216.58.211.109  192.168.178.27   
6         dropbox  0.019853          1  192.168.178.27  216.58.211.109   
7         dropbox  0.020131          1  192.168.178.27  216.58.211.109   
13        dropbox  0.026194          0  192.168.178.67  192.168.178.27   
14        dropbox  0.026197          0  192.168.178.67  192.168.178.27   
25        dropbox  0.043604          0  216.58.211.109  192.168.178.27   
26        dropbox  0.043886          0  216.58.211.109  192.168.178.27   
27        dropbox  0.043890          0  216.58.211.109  192.168.178.27   
28        dropbox  0.044534          1  192.168.178.27  216.58.211.109   
42        dropbox  0.069712          1  192.168.178.27  216.58.211.109   
70        dropbox  0.226609          0  192.168.178.67  192.168.178.27   
71        dropbox  0.226766          0

...

265           75      UDP    60756 → 443 Len=33  
./raw_data/day_three/facebook.com_1589928845.csv_fixed.csv read
   website-index      time  direction             src             dst  \
1       facebook  0.001287          0  216.58.211.109  192.168.178.27   
4       facebook  0.001727          0  216.58.211.109  192.168.178.27   
7       facebook  0.003583          1  192.168.178.27  216.58.211.109   
18      facebook  0.023283          1  192.168.178.27  216.58.211.109   
31      facebook  0.037974          0  216.58.211.109  192.168.178.27   
32      facebook  0.038382          1  192.168.178.27  216.58.211.109   
33      facebook  0.038641          1  192.168.178.27  216.58.211.109   
47      facebook  0.058815          0  216.58.211.109  192.168.178.27   
48      facebook  0.059129          0  216.58.211.109  192.168.178.27   
49      facebook  0.059131          0  216.58.211.109  192.168.178.27   
50      facebook  0.059673          1  192.168.178.27  216.58.211.109   
53      fa

   website-index      time  direction             src             dst  \
1       linkedin  0.011391          0  172.217.19.205  192.168.178.27   
2       linkedin  0.011581          0  172.217.19.205  192.168.178.27   
4       linkedin  0.013205          1  192.168.178.27  172.217.19.205   
12      linkedin  0.031221          1  192.168.178.27  172.217.19.205   
24      linkedin  0.052715          0  172.217.19.205  192.168.178.27   
27      linkedin  0.053151          1  192.168.178.27  172.217.19.205   
29      linkedin  0.053711          1  192.168.178.27  172.217.19.205   
43      linkedin  0.078643          0  172.217.19.205  192.168.178.27   
44      linkedin  0.078646          0  172.217.19.205  192.168.178.27   
45      linkedin  0.079018          1  192.168.178.27  172.217.19.205   
46      linkedin  0.079333          0  172.217.19.205  192.168.178.27   
54      linkedin  0.105126          1  192.168.178.27  172.217.19.205   

    packet size protocol                  info  
1

    website-index      time  direction             src              dst  \
1             cnn  0.002474          1  192.168.178.27   216.58.211.109   
9             cnn  0.013745          0  216.58.211.109   192.168.178.27   
10            cnn  0.014151          1  192.168.178.27   216.58.211.109   
11            cnn  0.014411          1  192.168.178.27   216.58.211.109   
29            cnn  0.039023          0  216.58.211.109   192.168.178.27   
30            cnn  0.040366          0  216.58.211.109   192.168.178.27   
31            cnn  0.040478          1  192.168.178.27   216.58.211.109   
540           cnn  2.190480          1  192.168.178.27  192.168.178.255   

     packet size protocol                  info  
1           1392      UDP  64184 → 443 Len=1350  
9           1392      UDP  443 → 64184 Len=1350  
10            75      UDP    64184 → 443 Len=33  
11           389      UDP   64184 → 443 Len=347  
29           883      UDP   443 → 64184 Len=841  
30           206      UD

   website-index      time  direction             src             dst  \
6      wikipedia  0.014554          1  192.168.178.27   172.217.17.45   
15     wikipedia  0.026603          0   172.217.17.45  192.168.178.27   
16     wikipedia  0.026996          1  192.168.178.27   172.217.17.45   
17     wikipedia  0.027294          1  192.168.178.27   172.217.17.45   
31     wikipedia  0.051214          0   172.217.17.45  192.168.178.27   
32     wikipedia  0.051218          0   172.217.17.45  192.168.178.27   
33     wikipedia  0.051550          0   172.217.17.45  192.168.178.27   
35     wikipedia  0.051619          1  192.168.178.27   172.217.17.45   
44     wikipedia  0.076728          1  192.168.178.27   172.217.17.45   

    packet size protocol                  info  
6          1392      UDP  57205 → 443 Len=1350  
15         1392      UDP  443 → 57205 Len=1350  
16           75      UDP    57205 → 443 Len=33  
17          389      UDP   57205 → 443 Len=347  
31          882      UDP

    website-index      time  direction             src             dst  \
8            hulu  0.012458          1  192.168.178.27  216.58.211.109   
17           hulu  0.026607          0  216.58.211.109  192.168.178.27   
18           hulu  0.027004          1  192.168.178.27  216.58.211.109   
19           hulu  0.027264          1  192.168.178.27  216.58.211.109   
37           hulu  0.054612          0  216.58.211.109  192.168.178.27   
38           hulu  0.054859          0  216.58.211.109  192.168.178.27   
39           hulu  0.054861          0  216.58.211.109  192.168.178.27   
40           hulu  0.055002          1  192.168.178.27  216.58.211.109   
41           hulu  0.080115          1  192.168.178.27  216.58.211.109   
267          hulu  3.958987          1  192.168.178.27  172.217.19.206   
274          hulu  3.977256          0  172.217.19.206  192.168.178.27   
275          hulu  3.978574          0  172.217.19.206  192.168.178.27   
276          hulu  3.979724          1